Loading the package we are going to use:

In [39]:
import numpy as np
from emo_utils import *
import emoji
import random
import math

import matplotlib.pyplot as plt

from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

%matplotlib inline

### 1 Dataset EMOJISET:

We have a tiny dataset (X, Y) where:

 - X contains 127 sentences (strings)
 - Y contains a integer label between 0 and 4 corresponding to an emoji for each sentence

In [3]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [4]:
maxLen = len(max(X_train, key=len).split())

Test loading:

In [5]:
print("Train:")
for index in range(len(Y_train)):
    print(index, X_train[index], label_to_emoji(Y_train[index]))
print("Test:")
for index in range(len(Y_test)):
    print(index, X_test[index], label_to_emoji(Y_test[index]))

Train:
0 never talk to me again 😞
1 I am proud of your achievements 😄
2 It is the worst day in my life 😞
3 Miss you so much ❤️
4 food is life 🍴
5 I love you mum ❤️
6 Stop saying bullshit 😞
7 congratulations on your acceptance 😄
8 The assignment is too long  😞
9 I want to go play ⚾
10 she did not answer my text  😞
11 Your stupidity has no limit 😞
12 how many points did he score ⚾
13 my algorithm performs poorly 😞
14 I got approved 😄
15 Stop shouting at me 😞
16 Sounds like a fun plan ha ha 😄
17 no one likes him 😞
18 the game just finished ⚾
19 I will celebrate soon 😄
20 So sad you are not coming 😞
21 She is my dearest love ❤️
22 Good job 😄
23 It was funny lol 😄
24 candy is life  😄
25 The chicago cubs won again ⚾
26 I am hungry 🍴
27 I am so excited to see you after so long 😄
28 you did well on you exam 😄
29 lets brunch some day 🍴
30 he is so cute ❤️
31 How dare you ask that 😞
32 do you want to join me for dinner  🍴
33 I said yes 😄
34 she is attractive ❤️
35 you suck 😞
36 she smiles a lot 

### 2  Overview of the Emojifier-V1:

Converting  𝑌  from its current shape current shape (𝑚,1) into a "one-hot representation" (𝑚,5):

In [6]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

Test `convert_to_one_hot` method:

In [7]:
print("convert_to_one_hot:")
for index in range(len(Y_train)):
    print(index, X_train[index], label_to_emoji(Y_train[index]), Y_oh_train[index])

convert_to_one_hot:
0 never talk to me again 😞 [0. 0. 0. 1. 0.]
1 I am proud of your achievements 😄 [0. 0. 1. 0. 0.]
2 It is the worst day in my life 😞 [0. 0. 0. 1. 0.]
3 Miss you so much ❤️ [1. 0. 0. 0. 0.]
4 food is life 🍴 [0. 0. 0. 0. 1.]
5 I love you mum ❤️ [1. 0. 0. 0. 0.]
6 Stop saying bullshit 😞 [0. 0. 0. 1. 0.]
7 congratulations on your acceptance 😄 [0. 0. 1. 0. 0.]
8 The assignment is too long  😞 [0. 0. 0. 1. 0.]
9 I want to go play ⚾ [0. 1. 0. 0. 0.]
10 she did not answer my text  😞 [0. 0. 0. 1. 0.]
11 Your stupidity has no limit 😞 [0. 0. 0. 1. 0.]
12 how many points did he score ⚾ [0. 1. 0. 0. 0.]
13 my algorithm performs poorly 😞 [0. 0. 0. 1. 0.]
14 I got approved 😄 [0. 0. 1. 0. 0.]
15 Stop shouting at me 😞 [0. 0. 0. 1. 0.]
16 Sounds like a fun plan ha ha 😄 [0. 0. 1. 0. 0.]
17 no one likes him 😞 [0. 0. 0. 1. 0.]
18 the game just finished ⚾ [0. 1. 0. 0. 0.]
19 I will celebrate soon 😄 [0. 0. 1. 0. 0.]
20 So sad you are not coming 😞 [0. 0. 0. 1. 0.]
21 She is my dearest love ❤

### 3 Implementing Emojifier-V1:

Convertind an input sentence into the word 50-dimensional GloVe vector representation:

We are loaded:

- `word_to_index`: dictionary mapping from words to their indices in the vocabulary (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

In [8]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

Text GloVo:

In [9]:
#I am always working
print("\nIndex:", word_to_index["i"], "\nWord:", "i", "\nVector:", word_to_vec_map["i"])
print("\nIndex:", word_to_index["am"], "\nWord:", "am", "\nVector:", word_to_vec_map["am"])
print("\nIndex:", word_to_index["always"], "\nWord:", "always", "\nVector:", word_to_vec_map["always"])
print("\nIndex:", word_to_index["working"], "\nWord:", "working", "\nVector:", word_to_vec_map["working"])


Index: 185457 
Word: i 
Vector: [ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-01  8.8180e-01  2.9561e-01
  1.3548e+00 -2.5701e+00 -1.3523e+00  4.5880e-01  1.0068e+00 -1.1856e+00
  3.4737e+00  7.7898e-01 -7.2929e-01  2.5102e-01 -2.6156e-01 -3.4684e-01
  5.5841e-01  7.5098e-01  4.9830e-01 -2.6823e-01 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01]

Index: 52943 
Word: am 
Vector: [ 0.34664   0.39805   0.4897   -0.51421   0.54574  -1.2005    0.32107
  0.74004  -1.4979   -0.19651  -0.12631  -0.37703  -0.62569   0.038792
  1.0579    0.77199  -0.18589   1.3032   -0.72128   0.40231   0.066442
  1.2315    0.93956   1.3903    1.5334   -1.473    -0.34997   0.31562
  0.90691   0.45498   2.5481    0.1641   -

`sentence_to_avg()` method convert every sentence to lower-case, then split the sentence into a list of words. For each word in the sentence, access its GloVe representation. Then, average all these values:

In [10]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    words = None
    words = sentence.lower()
    words = words.split()

    avg = list()
    for i in range(0, len(word_to_vec_map[words[0]])):
        avg.append(0)
    
    for i in range(0,len(words)):
        avg = avg + word_to_vec_map[words[i]]
    avg = avg/len(words)
        
    return avg

Test `sentence_to_avg()` method:

In [11]:
avg = sentence_to_avg("I am always working", word_to_vec_map)
print("avg = ", avg)

avg =  [ 0.2203125   0.1675675  -0.01825675 -0.5823375   0.5751325  -0.46347975
 -0.359695    0.36966125 -0.76170225 -0.05831811  0.00868975  0.31552
 -0.6267425   0.04421435  0.804925    0.398175    0.2166125   0.6795025
 -0.021271   -0.433325   -0.176527    0.857225    0.555932    0.6414575
  1.0257525  -1.83725    -0.6941275   0.138315    0.683832   -0.31363238
  3.1559      0.37279275 -0.50122075 -0.1706975  -0.2846885  -0.3585625
  0.2655025   0.43667     0.11652    -0.15790025  0.03425143 -0.29827532
  0.031       0.48315703  0.0288805   0.07005925 -0.3304275  -0.1512855
 -0.1623025   0.837725  ]


`Emojifier-V1` class:

In [67]:
# Input - Layer
model = Sequential()
model.add(Dense(5, input_dim=50, activation='sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

X = np.zeros((len(X_train), 50), dtype=float)
for i in range(0, len(X_train)):
    X[i] = sentence_to_avg(X_train[i], word_to_vec_map)

model.fit(X, Y_oh_train, epochs = 100, batch_size=10)

scores = model.evaluate(X, Y_oh_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/100
132/132 [==============================] - 3s 23ms/step - loss: 0.7403 - acc: 0.5030
Epoch 2/100
132/132 [==============================] - 0s 1ms/step - loss: 0.6802 - acc: 0.6076
Epoch 3/100
132/132 [==============================] - 0s 2ms/step - loss: 0.6315 - acc: 0.6682
Epoch 4/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5939 - acc: 0.7076
Epoch 5/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5645 - acc: 0.7348
Epoch 6/100
132/132 [==============================] - 0s 1ms/step - loss: 0.5424 - acc: 0.7591
Epoch 7/100
132/132 [==============================] - 0s 697us/step - loss: 0.5249 - acc: 0.7742
Epoch 8/100
132/132 [==============================] - 0s 1ms/step - loss: 0.5114 - acc: 0.7803
Epoch 9/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5007 - acc: 0.7924
Epoch 10/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4914 - acc: 0.7955
Epoch 11/100
132/132 [==============

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.115000). Check your callbacks.
  % delta_t_median)


Epoch 42/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3870 - acc: 0.8121
Epoch 43/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3847 - acc: 0.8136
Epoch 44/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3826 - acc: 0.8167
Epoch 45/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3806 - acc: 0.8167
Epoch 46/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3785 - acc: 0.8182
Epoch 47/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3765 - acc: 0.8182
Epoch 48/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3745 - acc: 0.8197
Epoch 49/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3726 - acc: 0.8212
Epoch 50/100
132/132 [==============================] - 0s 614us/step - loss: 0.3706 - acc: 0.8227
Epoch 51/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3687 - acc: 0.8242
Epoch 52/100
132/132 [======

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.103000). Check your callbacks.
  % delta_t_median)



132/132 [==============================] - 0s 682us/step - loss: 0.3163 - acc: 0.8667
Epoch 89/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3157 - acc: 0.8636
Epoch 90/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3141 - acc: 0.8636
Epoch 91/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3131 - acc: 0.8667
Epoch 92/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3126 - acc: 0.8682
Epoch 93/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3110 - acc: 0.8727
Epoch 94/100
132/132 [==============================] - 0s 674us/step - loss: 0.3100 - acc: 0.8682
Epoch 95/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3094 - acc: 0.8682
Epoch 96/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3079 - acc: 0.8712
Epoch 97/100
132/132 [==============================] - 0s 1ms/step - loss: 0.3069 - acc: 0.8727
Epoch 98/100
110/132 [================

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.132000). Check your callbacks.
  % delta_t_median)


132/132 [==============================] - 0s 2ms/step - loss: 0.3061 - acc: 0.8758
Epoch 99/100
132/132 [==============================] - 0s 894us/step - loss: 0.3050 - acc: 0.8773
Epoch 100/100
132/132 [==============================] - 1s 6ms/step

acc: 87.73%
